In [75]:
# plotting utilities
import seaborn as sns
import matplotlib.pyplot as plt

# tensorflow
import tensorflow as tf
import tensorflow.keras as K


# data-science & processing tools
import numpy as np
import pandas as pd
import sklearn.metrics
import h5py
import pygmnormalize as gmn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

### Getting the Data

In [29]:
df = pd.read_csv('raw_data.tsv', sep='\t', header=0)

In [30]:
gene_ID = df['Gene ID']
gene_names = df['Gene Name']

df.drop('Gene ID', axis=1, inplace=True)
df.drop('Gene Name', axis=1, inplace=True)

In [31]:
normal_df = gmn.total_count_normalization(df)

In [32]:
X = normal_df.values

In [34]:
X.shape

(65217, 322)

In [35]:
## indices where values are greater than zero
idx2 = np.where(X.any(axis=1))[0]
newX = X[idx2]
newX.shape

(54914, 322)

In [37]:
## get rows where a value is greater than 0.0001
rows = np.unique(np.where(newX > 0.0001)[0])
newerX = newX[rows]
newerX.shape

(5637, 322)

In [38]:
names = gene_names[idx2].values[rows]

In [39]:
by_patient = newerX.T
by_patient.shape

(322, 5637)

In [82]:
classes = open('classifications.txt', 'r').read()
classes = eval(classes)
y = np.array(list(classes.values()))

In [83]:
label_enc = LabelEncoder()
labels = label_enc.fit_transform(y)
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0,
       2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,

In [84]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [89]:
labels = labels.reshape(len(labels), 1)
one_hot = enc.fit_transform(labels)
one_hot

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [90]:
X_train, X_test, y_train, y_test = train_test_split(by_patient, one_hot, test_size=0.33, random_state=42)

In [91]:
X_train.shape

(215, 5637)

### Model

In [92]:
model = K.Sequential()
model.add(K.layers.Dense(units=10, activation='relu', input_shape=(5637,), 
                         kernel_regularizer=K.regularizers.l2(l=0.001)))
model.add(K.layers.Dropout(rate=0.001))
model.add(K.layers.Dense(units=200, activation='relu', kernel_regularizer=K.regularizers.l2(l=0.001)))
model.add(K.layers.Dense(units=3, activation='softmax'))

In [93]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 10)                56380     
_________________________________________________________________
dropout_6 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 200)               2200      
_________________________________________________________________
dense_30 (Dense)             (None, 3)                 603       
Total params: 59,183
Trainable params: 59,183
Non-trainable params: 0
_________________________________________________________________


In [94]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [95]:
model.fit(X_train, y_train, epochs=100, batch_size=10)

Train on 215 samples
Epoch 1/100
215/215 [==============================] - 1s 5ms/sample - loss: 0.6416 - accuracy: 0.6667
Epoch 2/100
215/215 [==============================] - 0s 417us/sample - loss: 0.5758 - accuracy: 0.6915
Epoch 3/100
215/215 [==============================] - 0s 412us/sample - loss: 0.5290 - accuracy: 0.7798
Epoch 4/100
215/215 [==============================] - 0s 406us/sample - loss: 0.5164 - accuracy: 0.7798
Epoch 5/100
215/215 [==============================] - 0s 403us/sample - loss: 0.5166 - accuracy: 0.7798
Epoch 6/100
215/215 [==============================] - 0s 402us/sample - loss: 0.5119 - accuracy: 0.7798
Epoch 7/100
215/215 [==============================] - 0s 403us/sample - loss: 0.5102 - accuracy: 0.7798
Epoch 8/100
215/215 [==============================] - 0s 406us/sample - loss: 0.5098 - accuracy: 0.7798
Epoch 9/100
215/215 [==============================] - 0s 403us/sample - loss: 0.5089 - accuracy: 0.7798
Epoch 10/100
215/215 [==============

215/215 [==============================] - 0s 549us/sample - loss: 0.4587 - accuracy: 0.8016
Epoch 79/100
215/215 [==============================] - 0s 517us/sample - loss: 0.4626 - accuracy: 0.8109
Epoch 80/100
215/215 [==============================] - 0s 478us/sample - loss: 0.4576 - accuracy: 0.8062
Epoch 81/100
215/215 [==============================] - 0s 477us/sample - loss: 0.4545 - accuracy: 0.8031
Epoch 82/100
215/215 [==============================] - 0s 465us/sample - loss: 0.4562 - accuracy: 0.8000
Epoch 83/100
215/215 [==============================] - 0s 463us/sample - loss: 0.4546 - accuracy: 0.8186
Epoch 84/100
215/215 [==============================] - 0s 465us/sample - loss: 0.4538 - accuracy: 0.8031
Epoch 85/100
215/215 [==============================] - 0s 469us/sample - loss: 0.4565 - accuracy: 0.8155
Epoch 86/100
215/215 [==============================] - 0s 481us/sample - loss: 0.4514 - accuracy: 0.8124
Epoch 87/100
215/215 [==============================] - 0s 